## Check environment variables

Make sure that the environment variables that stac_ipyleaflet depends on are available

In [1]:
import os

print(
    os.environ['TITILER_STAC_ENDPOINT'],
    os.environ['TITILER_ENDPOINT'],
    os.environ['STAC_CATALOG_NAME'],
    os.environ['STAC_CATALOG_URL'],
    os.environ['STAC_BROWSER_URL'],
    sep="\n",
)

https://openveda.cloud/api/stac
https://openveda.cloud/api/raster
VEDA STAC
https://openveda.cloud/api/stac
https://openveda.cloud/


## Test xarray

- [x] import library
- [x] load data
- [x] resample

In [2]:
import xarray as xr
import xarray.tutorial

air = xr.tutorial.load_dataset("air_temperature")
print(air)

<xarray.Dataset> Size: 31MB
Dimensions:  (time: 2920, lat: 25, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...


In [3]:
monthly = air.air.resample({"time": "ME"}).mean()
print(monthly.min())

<xarray.DataArray 'air' ()> Size: 8B
array(231.72116071)


## Test geopandas

- [x] import library
- [x] read data from remote
- [x] render using shapely
- [x] render using folium

In [4]:
import geopandas as gpd

In [5]:
URL = "https://geodata.ucdavis.edu/gadm/gadm4.1/json/gadm41_FRA_0.json"
df = gpd.read_file(URL)

In [6]:
df.geometry[0].bounds

(-5.1438, 41.3338, 9.5604, 51.0894)

## Test fsspec

- [x] import library
- [x] `ls` public s3 bucket

In [7]:
import fsspec

In [8]:
fs = fsspec.filesystem("s3", anon=True)

fs.ls("s3://sentinel-inventory/sentinel-s2-l1c/")

['sentinel-inventory/sentinel-s2-l1c/inventory-orc',
 'sentinel-inventory/sentinel-s2-l1c/sentinel-s2-l1c-inventory']

## Test PySTAC-client

- [x] import library
- [x] open catalog
- [x] search catalog

In [9]:
from pystac_client import Client

In [10]:
STAC_API_URL = "https://staging.openveda.cloud/api/stac/"
catalog = Client.open(STAC_API_URL)
print(catalog)  # don't want the full repr here since it will change everytime anything is added or removed from STAC

<Client id=stac-fastapi>


In [11]:
items = list(catalog.search(collections=["social-vulnerability-index-overall-nopop"], datetime="2000").items())
items

[<Item id=svi_2000_tract_overall_wgs84_nopop_cog>]

## Test hvplot

- [x] import library
- [x] geo plot
- [x] non-geo plot

In [12]:
import hvplot.xarray

In [13]:
p = monthly.hvplot(geo=True, x="lon", y="lat", dynamic=False)
print(p)

:HoloMap   [time]
   :Image   [lon,lat]   (4xDaily Air temperature at sigma level 995)


In [14]:
# Data that goes into plot for May
print(p.values()[4].data)

<xarray.Dataset> Size: 11kB
Dimensions:  (lat: 25, lon: 53, time: 1)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 8B 2013-05-31
Data variables:
    air      (time, lat, lon) float64 11kB 266.6 266.8 266.8 ... 297.1 297.1


In [15]:
p = monthly.sel(lat=40, lon=285).hvplot()
print(p)

:Curve   [time]   (4xDaily Air temperature at sigma level 995)


In [16]:
print(p.data)

             lat    lon         air
time                               
2013-01-31  40.0  285.0  274.552661
2013-02-28  40.0  285.0  273.603929
2013-03-31  40.0  285.0  276.138871
2013-04-30  40.0  285.0  282.911333
2013-05-31  40.0  285.0  288.135484
2013-06-30  40.0  285.0  293.957167
2013-07-31  40.0  285.0  297.506210
2013-08-31  40.0  285.0  295.102258
2013-09-30  40.0  285.0  291.740083
2013-10-31  40.0  285.0  288.149839
2013-11-30  40.0  285.0  279.911917
2013-12-31  40.0  285.0  276.186048
2014-01-31  40.0  285.0  270.596210
2014-02-28  40.0  285.0  272.124018
2014-03-31  40.0  285.0  274.704355
2014-04-30  40.0  285.0  282.099000
2014-05-31  40.0  285.0  288.787742
2014-06-30  40.0  285.0  294.042000
2014-07-31  40.0  285.0  295.739032
2014-08-31  40.0  285.0  295.084677
2014-09-30  40.0  285.0  293.141000
2014-10-31  40.0  285.0  288.132177
2014-11-30  40.0  285.0  279.517667
2014-12-31  40.0  285.0  277.252258


## Test virtualizarr + icechunk

Based off of [virtualizarr docs](https://virtualizarr.readthedocs.io/en/stable/index.html#quick-usage-example)

- [x] import libraries
- [x] virtualize data from s3
- [x] store virtual zarr store as icechunk

In [17]:
import icechunk
import obstore

from virtualizarr import open_virtual_dataset
from virtualizarr.parsers import HDFParser
from virtualizarr.registry import ObjectStoreRegistry

In [18]:
bucket = "s3://nex-gddp-cmip6"
path = "NEX-GDDP-CMIP6/ACCESS-CM2/ssp126/r1i1p1f1/tasmax/tasmax_day_ACCESS-CM2_ssp126_r1i1p1f1_gn_2015_v2.0.nc"
store = obstore.store.from_url(bucket, region="us-west-2", skip_signature=True)

registry = ObjectStoreRegistry({bucket: store})

parser = HDFParser()
vds = open_virtual_dataset(
  url=f"{bucket}/{path}",
  parser=parser,
  registry=registry,
  loadable_variables=[],
)
print(vds)

<xarray.Dataset> Size: 1GB
Dimensions:  (time: 365, lat: 600, lon: 1440)
Coordinates:
    time     (time) float64 3kB ManifestArray<shape=(365,), dtype=float64, ch...
    lat      (lat) float64 5kB ManifestArray<shape=(600,), dtype=float64, chu...
    lon      (lon) float64 12kB ManifestArray<shape=(1440,), dtype=float64, c...
Data variables:
    tasmax   (time, lat, lon) float32 1GB ManifestArray<shape=(365, 600, 1440...
Attributes: (12/22)
    cmip6_source_id:       ACCESS-CM2
    cmip6_institution_id:  CSIRO-ARCCSS
    cmip6_license:         CC-BY-SA 4.0
    activity:              NEX-GDDP-CMIP6
    Conventions:           CF-1.7
    frequency:             day
    ...                    ...
    doi:                   https://doi.org/10.7917/OFSG3345
    external_variables:    areacella
    contact:               Dr. Bridget Thrasher: bridget@climateanalyticsgrou...
    creation_date:         Sat Nov 16 13:31:18 PST 2024
    disclaimer:            These data are considered provisional

/srv/conda/envs/notebook/lib/python3.12/site-packages/numcodecs/zarr3.py:145: UserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


In [19]:
config = icechunk.RepositoryConfig.default()
container = icechunk.VirtualChunkContainer(
    url_prefix="s3://nex-gddp-cmip6/",
    store=icechunk.s3_store(region="us-west-2", anonymous=True),
)
config.set_virtual_chunk_container(container)

icechunk_store = icechunk.in_memory_storage()
repo = icechunk.Repository.create(icechunk_store, config)
session = repo.writable_session("main")
vds.vz.to_icechunk(session.store)
session.commit("Create virtual store");